# **주요 내용**

*   

In [1]:
notebookName = "20200529"
notebookName

'20200529'

In [2]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16426102289983380989, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5569114109258626943
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5035351099359405122
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15701463552
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6940979240236665035
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [4]:
import platform
platform.platform()

'Linux-4.19.104+-x86_64-with-Ubuntu-18.04-bionic'

In [5]:
!nvidia-smi

Fri May 29 12:55:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    32W / 250W |    353MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## **Unzip & Format Change**

In [0]:
import os

kor_pcm_path = "./kor_pcm_dataset"
kor_wav_path = "./kor_wav_dataset"

eng_pcm_path = "./eng_pcm_dataset"
eng_wav_path = "./eng_wav_dataset"

# mkdir
if not os.path.exists(kor_pcm_path): os.mkdir(kor_pcm_path)
if not os.path.exists(kor_wav_path): os.mkdir(kor_wav_path)

if not os.path.exists(eng_pcm_path): os.mkdir(eng_pcm_path)
if not os.path.exists(eng_wav_path): os.mkdir(eng_wav_path)

# unsip
!unzip -q "/content/drive/My Drive/ETRI/9_voice_download_4_Korean_English_by_Korean_part2.zip" -d "./kor_pcm_dataset"
!unzip -q "/content/drive/My Drive/ETRI/8_voice_download_4_Korean_English_by_Korean_part1.zip" -d "./eng_pcm_dataset"

In [8]:
print(os.listdir(kor_pcm_path)[:5], len(os.listdir(kor_pcm_path)))
print(os.listdir(eng_pcm_path)[:5], len(os.listdir(eng_pcm_path)))

['GSM07355MIW0', 'GSF04122LHK0', 'GSF04923PEY0', 'I4F03543KSY0', 'GSM06235BJH0'] 50
['GSM07355MIW0', 'GSF04122LHK0', 'GSF04923PEY0', 'I4F03543KSY0', 'GSM06235BJH0'] 50


In [0]:
import wave

kor_labels = dict()
eng_labels = dict()

for pcm_path, wav_path in zip([kor_pcm_path, eng_pcm_path], [kor_wav_path, eng_wav_path]):
    tmp_dict = dict()

    for i, folder_path in enumerate(sorted(os.listdir(pcm_path))):
        tmp_dict[folder_path] = i

        file_list = os.listdir(os.path.join(pcm_path, folder_path)) # label's path
        file_list_pcm = [files for files in file_list if files.endswith(".pcm")] # *.pcm files

        for j, file_path in enumerate(sorted(file_list_pcm)):
            with open(os.path.join(pcm_path, folder_path, file_path), "rb") as pcm_file:
                pcm_data = pcm_file.read()
            # save as {label}_{features}.wav format
            with wave.open(os.path.join(wav_path, "{}_{}.wav".format(i, j)), 'wb') as wav_file:
                wav_file.setparams((1, 2, 16000, 0, 'NONE', 'NONE')) # 16kHz, mono channel
                wav_file.writeframes(pcm_data)

    # save labeling pairs
    if pcm_path == kor_pcm_path: 
        kor_labels = tmp_dict
    else:
        eng_labels = tmp_dict

In [10]:
# Check that two groups are equal
set(kor_labels.keys()) == set(eng_labels.keys())

True

In [11]:
# Check that the order of two groups are equal
list(kor_labels.keys()) == list(eng_labels.keys())

True

In [0]:
import numpy as np

num_labels = 50
num_features = 100
data_length = 3 ** 9

## **Split Train Data, Test Data**

In [96]:
from scipy.io import wavfile

data = np.zeros((2, num_labels , num_features, data_length)) # size: (2, 50, 100, 3^9)
labels = np.zeros((2, num_labels, num_features))                    # size: (2, 50, 100)

for i, wav_path in enumerate([kor_wav_path, eng_wav_path]):
    for j in range(num_labels):
        for k in range(num_features):
            file_name = "{}_{}.wav".format(j, k)
            _, samples = wavfile.read(os.path.join(wav_path, file_name))
            
            start = int(len(samples) - data_length) // 2
            end = start + data_length
            
            data[i, j, k] = samples[start:end]
            labels[i, j, k] = j

# save       
np.save("./data", data)
np.save("./labels", labels)

print("data.shape: {}, labels.shape: {}".format(data.shape, labels.shape))

data.shape: (2, 50, 100, 19683), labels.shape: (2, 50, 100)


In [97]:
num_train_data = 5

choosen_label_list = np.random.choice(num_labels, size = 5, replace = False)

print("Choosen labels: {}".format(choosen_label_list))

Choosen labels: [ 6 35 21 48 47]


In [135]:
train_data_list = [
    np.reshape(data[0, choosen_label_list[:, np.newaxis], np.arange(5), :], (-1, data_length)), # only Korean
    np.reshape(data[1, choosen_label_list[:, np.newaxis], np.arange(5), :], (-1, data_length)), # only English
    np.concatenate((                                                                                                                             # mixed (rate = 3:2)
        np.reshape(data[0, choosen_label_list[:, np.newaxis], np.random.choice(5, size = 3, replace = False), :], (-1, data_length)),
        np.reshape(data[1, choosen_label_list[:, np.newaxis], np.random.choice(5, size = 2, replace = False), :], (-1, data_length))),
        axis = 0
    )]

train_labels_list = [
    np.reshape(labels[0, choosen_label_list[:, np.newaxis], np.arange(5)], (-1)), # only Korean
    np.reshape(labels[1, choosen_label_list[:, np.newaxis], np.arange(5)], (-1)), # only English
    np.concatenate((                                                                                                       # mixed (rate = 3:2)
        np.reshape(labels[0, choosen_label_list[:, np.newaxis], np.random.choice(5, size = 3, replace = False)], (-1)),
        np.reshape(labels[1, choosen_label_list[:, np.newaxis], np.random.choice(5, size = 2, replace = False)], (-1))),
        axis = 0
    )]

print("train_data_list.shape: {}".format([a.shape for a in train_data_list]))
print("train_labels_list.shape: {}".format([a.shape for a in train_labels_list]))

train_data_list.shape: [(25, 19683), (25, 19683), (25, 19683)]
train_labels_list.shape: [(25,), (25,), (25,)]


In [136]:
test_data = np.reshape(data[:, choosen_label_list[:, np.newaxis], np.arange(5, 100, 1), :], (-1, data_length))
test_labels = np.reshape(labels[:, choosen_label_list[:, np.newaxis], np.arange(5, 100, 1)], (-1))

print("test_data.shape: {}".format(test_data.shape))
print("test_labels.shape: {}".format(test_labels.shape))

test_data.shape: (950, 19683)
test_labels.shape: (950,)


## **Build Model**

In [0]:
# from tensorflow.keras.utils import plot_model

def build_model(model_name, activation):
    if activation not in ["softmax", None]:
        print("Wrong activation.. Choose softmax or None.")
        return

    model_input = tf.keras.layers.Input((data_length, 1))

    x = tf.keras.layers.Conv1D(name = "block1_conv", filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')(model_input)
    x = tf.keras.layers.BatchNormalization(name = "block1_bn")(x)
    x = tf.keras.layers.MaxPooling1D(name = "block1_pool", pool_size = 3)(x)
    x = tf.keras.layers.Dropout(name = "block1_dropout", rate = 0.3)(x)

    x = tf.keras.layers.Conv1D(name = "block2_conv", filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')(x)
    x = tf.keras.layers.BatchNormalization(name = "block2_bn")(x)
    x = tf.keras.layers.MaxPooling1D(name = "block2_pool", pool_size = 3)(x)
    x = tf.keras.layers.Dropout(name = "block2_dropout", rate = 0.3)(x)

    x = tf.keras.layers.Conv1D(name = "block3_conv", filters = 128, kernel_size = 3, padding = 'same', activation = 'relu')(x)
    x = tf.keras.layers.BatchNormalization(name = "block3_bn")(x)
    x = tf.keras.layers.MaxPooling1D(name = "block3_pool", pool_size = 3)(x)
    x = tf.keras.layers.Dropout(name = "block3_dropout", rate = 0.3)(x)

    x = tf.keras.layers.Conv1D(name = "block4_conv", filters = 256, kernel_size = 3, padding = 'same', activation = 'relu')(x)
    x = tf.keras.layers.BatchNormalization(name = "block4_bn")(x)
    x = tf.keras.layers.MaxPooling1D(name = "block4_pool", pool_size = 3)(x)
    x = tf.keras.layers.Dropout(name = "block4_dropout", rate = 0.3)(x)

    x = tf.keras.layers.GlobalAveragePooling1D(name = "avg_pool")(x)
    x = tf.keras.layers.Dense(name = "dense", units = 256, activation = activation)(x)

    out = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis = 1), name = "l2_norm_embedding")(x) # L2 normalize embeddings

    model = tf.keras.models.Model(
        inputs = model_input,
        outputs = out if activation == None else x,
        name = model_name)
    
    # model.summary()
    # plot_model(model, show_shapes = True, to_file = 'model.png')

    return model

## **Compile and Train model**

In [0]:
import tensorflow_addons as tfa

# callback
def scheduler(epoch, lr):
    # lr = 0.3 * lr for every 50 epochs
    if not epoch:
        return lr
    return 0.3 * lr if not (epoch % 50) else lr

def compile_and_train_model(model, epochs, train_dataset):
    model.compile(
        optimizer = tf.keras.optimizers.Adam(0.01),
        loss = tfa.losses.TripletSemiHardLoss())

    # Train the network
    history = model.fit(
        train_dataset,
        verbose = 0,
        epochs = epochs,
        callbacks = [tf.keras.callbacks.LearningRateScheduler(scheduler)])

    return history

In [0]:
batch_size = 128
epochs = 100
model_names = [
    ["train_kor_act_softmax", "train_kor_act_None"],
    ["train_eng_act_softmax", "train_eng_act_None"],
    ["train_mixed_act_softmax", "train_mixed_act_None"]]

models = []
histories = []

for i, (train_data, train_labels) in enumerate(zip(train_data_list, train_labels_list)):
    for j, act in enumerate(["softmax", None]):
        train_dataset = tf.data.Dataset.from_tensor_slices((train_data[:, :, np.newaxis], train_labels))
        train_dataset = train_dataset.shuffle(buffer_size = data_length * batch_size).batch(batch_size) # Of courve, 25 << 128 (restrictly small)

        model = build_model(model_names[i][j], act)
        history = compile_and_train_model(model, epochs, train_dataset)

        models.append(model)
        histories.append(history)

In [141]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_data[:, :, np.newaxis], test_labels))
test_dataset = test_dataset.batch(128)

test_dataset

<BatchDataset shapes: ((None, 19683, 1), (None,)), types: (tf.float64, tf.float64)>

In [0]:
# train_dataset = tf.data.Dataset.from_tensor_slices((kor_train_data[:, :, np.newaxis], kor_train_labels))
# test_dataset = tf.data.Dataset.from_tensor_slices((kor_test_data[:, :, np.newaxis], kor_test_labels))

# train_dataset = train_dataset.shuffle(buffer_size = data_length * 128 * 10).batch(128)
# test_dataset = test_dataset.batch(128)

In [0]:
# train_kor_test_kor_model_history.history.keys()

In [0]:
# from matplotlib import pyplot as plt
# plt.plot(np.arange(1, 100 + 1, 1), train_kor_test_kor_model_history.history['loss'])

## **Calculate Performance Evaluation Metrics (of test dataset)**
*   True Positive Rate (TPR)
*   False Positive Rate (FPR)
*   Receiver Operating Characteristic (ROC) Curve
*   Area Under the Curve (AUC)
*   Equal Error Rate (EER)

In [0]:
test_pred_list = []
test_true = test_labels

for model in models:
    if "act_softmax" in model.name:
        test_pred_list.append(tf.keras.models.Model(inputs = model.input, outputs = model.layers[-2].output).predict(test_dataset))
    else:
        test_pred_list.append(model.predict(test_dataset))

In [0]:
def calculate_tpr_fpr(test_pred, test_true):
    true_accept = np.zeros((0), dtype = float)
    false_accept = np.zeros((0), dtype = float)

    for l in choosen_label_list:
        same_class = test_pred[np.where(test_true == l)]
        diff_class = test_pred[np.where(test_true != l)]

        partial_true_accept = tf.norm([i[0] - i[1] for i in np.array(list(itertools.permutations(same_class, 2)))], axis = -1)
        partial_false_accept = tf.norm([i - j for i in same_class for j in diff_class], axis = -1)

        true_accept = np.concatenate([true_accept, partial_true_accept.numpy()], axis = None)
        false_accept = np.concatenate([false_accept, partial_false_accept.numpy()], axis = None)

        # print("true_accept.shape: {}".format(true_accept.shape))    # (num_features P 2) * num_class
        # print("false_accept.shape: {}".format(false_accept.shape)) # (num_features * (num_features * (num_class - 1))) * num_class

    tpr = np.zeros((10 ** 4), dtype = float)
    fpr = np.zeros((10 ** 4), dtype = float)

    total_results = np.concatenate([true_accept, false_accept], axis = None)

    # Strictly, below comparisons can be performed with all possible d values in total_results.
    # E.g. for i, d in enumerate(np.sort(np.concatenate([true_accept, false_accept], axis = None))):
    for i, d in enumerate(np.linspace(np.min(total_results), np.max(total_results), 10 ** 4)):    
        tpr[i] = np.where(true_accept <= d)[0].shape[0]
        fpr[i] = np.where(false_accept <= d)[0].shape[0]

    tpr = tpr / len(true_accept)
    fpr = fpr / len(false_accept)

    return tpr, fpr

In [0]:
tpr_fpr_dict = dict()

for model, test_pred in zip(models, test_pred_list):
    tpr_fpr_dict[model.name] = calculate_tpr_fpr(test_pred, test_true)

In [0]:
# import itertools

# true_accept = np.zeros((0), dtype = float)
# false_accept = np.zeros((0), dtype = float)

# for i in range(num_labels):
#     same_class = test_pred[np.where(test_true == i)]
#     diff_class = test_pred[np.where(test_true != i)]

#     partial_true_accept = tf.norm([i[0] - i[1] for i in np.array(list(itertools.permutations(same_class, 2)))], axis = -1)
#     partial_false_accept = tf.norm([i - j for i in same_class for j in diff_class], axis = -1)

#     true_accept = np.concatenate([true_accept, partial_true_accept.numpy()], axis = None)
#     false_accept = np.concatenate([false_accept, partial_false_accept.numpy()], axis = None)

In [0]:
# print("true_accept.shape: {}".format(true_accept.shape))    # (num_features P 2) * num_class
# print("false_accept.shape: {}".format(false_accept.shape)) # (num_features * (num_features * (num_class - 1))) * num_class

In [0]:
# tpr = np.zeros((10 ** 4), dtype = float)
# fpr = np.zeros((10 ** 4), dtype = float)

# total_results = np.concatenate([true_accept, false_accept], axis = None)

# # Strictly, below comparisons can be performed with all possible d values in total_results.
# # E.g. for i, d in enumerate(np.sort(np.concatenate([true_accept, false_accept], axis = None))):
# for i, d in enumerate(np.linspace(np.min(total_results), np.max(total_results), 10 ** 4)):    
#     tpr[i] = np.where(true_accept <= d)[0].shape[0]
#     fpr[i] = np.where(false_accept <= d)[0].shape[0]

In [0]:
# tpr = tpr / len(true_accept)
# fpr = fpr / len(false_accept)

In [0]:
# from bokeh.plotting import figure, show, output_notebook
# from bokeh.layouts import gridplot
# from bokeh.io import export_png
# from bokeh.colors import RGB

# output_notebook()

# p = figure(width = 400, height = 400, title = "False Potivie Rate and False Negative Rate")

# p.line(
#     np.linspace(0, 1, 10 ** 4), fpr,
#     color = "firebrick",
#     legend_label ='False Positive Rate',
#     line_width = 2
# )

# p.line(
#     np.linspace(0, 1, 10 ** 4), 1 - tpr,
#     color = "cornflowerblue",
#     legend_label ='False Negative Rate',
#     line_width = 2,
#     line_dash = (5, 3)
# )

# p.legend.location = "top_center"

# show(p)

In [0]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
from bokeh.io import export_png

from sklearn.metrics import auc

output_notebook()

def draw_roc_curve(tpr_fpr_dict):
    colors = 'firebrick deeppink deepskyblue darkslateblue forestgreen rosybrown'.split() # six lines

    p = figure(width = 500, height = 500, title = "Receiver Operating Characteristic Curve")

    for i, (key, values) in enumerate(tpr_fpr_dict.items()):
        p.line(
            values[1], values[0],
            line_color = colors[i],
            legend_label = "{} (AUC = {:.4f})".format(key, auc(values[1], values[0])),
            line_width = 2)
        
    p.line(
        np.linspace(0, 1, 10 ** 4), np.linspace(0, 1, 10 ** 4),
        line_color = "lightslategray",
        legend_label = "truely random (AUC = 0.5000)",
        line_width = 2,
        line_dash = (5, 3)
    )

    p.xaxis.axis_label = 'False Positive Rate'
    p.yaxis.axis_label = 'True Positive Rate'
    p.legend.location = "bottom_right"
    p.legend.label_text_font_size = '10pt'

    show(p)

In [255]:
draw_roc_curve(tpr_fpr_dict)

In [0]:
from scipy.optimize import brentq
from scipy.interpolate import interp1d

def calculate_eer(tpr, fpr):
    eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    return eer

In [0]:
def draw_det_curve(tpr_fpr_dict):
    colors = 'firebrick deeppink deepskyblue darkslateblue forestgreen rosybrown'.split() # six lines

    p = figure(width = 500, height = 500, title = "Detection Error Trade-off Curve")

    for i, (key, values) in enumerate(tpr_fpr_dict.items()):
        p.line(
            values[1], 1 - values[0],
            line_color = colors[i],
            legend_label = "{} (EER = {:.4f})".format(key, calculate_eer(values[0], values[1])),
            line_width = 2)
        
    p.line(
        np.linspace(0, 1, 10 ** 4), np.linspace(0, 1, 10 ** 4),
        line_color = "lightslategray",
        legend_label = "EER determination line",
        line_width = 2,
        line_dash = (5, 3)
    )

    p.xaxis.axis_label = 'False Match Rate'
    p.yaxis.axis_label = 'False Non-Match Rate'
    p.legend.location = "top_right"
    p.legend.label_text_font_size = '10pt'

    show(p)

In [262]:
draw_det_curve(tpr_fpr_dict)

In [0]:
from scipy.optimize import brentq
from scipy.interpolate import interp1d

def calculate_eer(tpr, fpr):
    eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    return eer

In [205]:
for (key, values) in tpr_fpr_dict.items():
    eer = calculate_eer(values[0], values[1])
    print("Model: {:<25} EER: {:<20.4f}".format(key, eer))

Model: train_kor_act_softmax     EER: 0.3223              
Model: train_kor_act_None        EER: 0.2777              
Model: train_eng_act_softmax     EER: 0.3223              
Model: train_eng_act_None        EER: 0.2221              
Model: train_mixed_act_softmax   EER: 0.3441              
Model: train_mixed_act_None      EER: 0.2416              


In [0]:
# # Save tpr, fpr
# _path = os.path.join("/content/drive/My Drive/results", notebookName)
# if not os.path.exists(_path): 
#     os.mkdir(_path)

# np.savez_compressed(
#     os.path.join(_path, notebookName),
#     tpr = tpr,
#     fpr = fpr
# )

## **Embedding Visualization with t-Stochastic Neighbor Embedding (t-SNE)**

*   https://lovit.github.io/nlp/representation/2018/09/28/tsne/
*   https://lovit.github.io/nlp/representation/2018/09/28/tsne/


In [0]:
from bokeh.models import Legend
from sklearn.manifold import TSNE

def draw_figure(x, y, title, show_figure = True,
    colors = 'firebrick darksalmon lightslategray darkslateblue forestgreen'.split()):

    p = figure(width = 400, height = 400, title = title)
    for i, c in enumerate(choosen_label_list):
        idx = np.where(y == c)[0]
        x_ = x[idx]
        p.scatter(
            x_[:,0], x_[:,1], fill_color = colors[i], line_color = colors[i], legend_label = str(c))
    if show_figure:
        show(p)
    return p

In [228]:
plots = []
for test_pred, model in zip(test_pred_list, models):
    y = TSNE(
        n_components = 2,
        perplexity = 30
    ).fit_transform(test_pred)
    plots.append(draw_figure(y, test_true, "{} (t-SNE with perplexity = 30)".format(model.name), show_figure = False))

gp = gridplot([
               [plots[0], plots[1]],
               [plots[2], plots[3]], 
               [plots[4], plots[5]]])

show(gp)

In [0]:
# import numpy as np

# n_data_per_class = num_train_data
# n_classes = num_labels

# x = train_kor_test_kor_model.predict(test_dataset)
# y = kor_test_labels

In [0]:
# from bokeh.plotting import figure, show, output_notebook
# from bokeh.layouts import gridplot
# from bokeh.io import export_png
# from bokeh.colors import RGB

# output_notebook()

# title = '{} classes {} points'.format(n_classes, n_classes * n_data_per_class)

# def draw_figure(x, y, title, show_figure=True,
#     colors = 'firebrick darksalmon lightslategray darkslateblue forestgreen'.split()):

#     p = figure(width = 400, height = 400, title = title)
#     for c in range(n_classes):
#         idx = np.where(y == c)[0]
#         x_ = x[idx]
#         p.scatter(x_[:,0], x_[:,1], fill_color = colors[c], line_color = colors[c])
#     if show_figure:
#         show(p)
#     return p

# p = draw_figure(x, y, title)

In [0]:
# # Predict test_set
# from sklearn.manifold import TSNE

# plots = []
# for perplexity in [2, 5, 10, 30, 100, 200, 500, 1000]:
#     z = TSNE(
#         n_components = 2,
#         perplexity = perplexity
#     ).fit_transform(x)
#     title = 't-SNE with perplexity = {}'.format(perplexity)
#     plots.append(draw_figure(z, y, title, show_figure = False))

# gp = gridplot([
#                [plots[0], plots[1]],
#                [plots[2], plots[3]], 
#                [plots[4], plots[5]],
#                [plots[6], plots[7]]])
# show(gp)

## **K-Nearest Neighbor Algorithm**

In [0]:
# from sklearn.neighbors import KNeighborsClassifier

# x = model.predict(train_dataset)
# # x = TSNE(n_components = 2).fit_transform(model.predict(train_dataset))
# y = train_labels

# classifier = KNeighborsClassifier(n_neighbors = 3, weights = 'distance')
# classifier.fit(x, y)

In [0]:
# _x = model.predict(test_dataset)
# # _x = TSNE(n_components = 2).fit_transform(model.predict(test_dataset))
# _y = test_labels

# print(classifier.score(_x, _y))

In [0]:
# model.predict(train_dataset).shape